In [53]:
using CSV
using DataFrames
using DataStructures
using Distributions
using GZip
using JSON
using LinearAlgebra
using Logging
using NPZ
using ProgressMeter
using PyPlot
using PyCall

using Distributions
using Random

import DataStructures.compare

include("julia/src/enums.jl")
include("julia/src/events.jl")

struct Earlier end
compare(c::Earlier, x::AbstractEvent, y::AbstractEvent) = time(x) < time(y)

include("julia/src/simstate.jl")
include("julia/src/progression.jl")
include("julia/src/simparams.jl")


include("julia/src/utils.jl")
include("julia/src/data_loading.jl")

include("julia/src/infection_kernels.jl")

include("julia/src/event_execution.jl")


execute! (generic function with 8 methods)

In [41]:
function initialize(rng=MersenneTwister(0);
        population_path::AbstractString,
        incubation_time_samples_path::AbstractString,
        t0_to_t1_samples_path::AbstractString,
        t0_to_t2_samples_path::AbstractString)
    
    individuals_df = load_individuals(population_path)
    num_individuals = individuals_df |> nrow

    dist_severity = Categorical([4/10, 3/10, 2/10, 1/10])
    dist_incubation_time = load_dist_from_samples(incubation_time_samples_path)
    dist_symptom_onset_time = load_dist_from_samples(t0_to_t1_samples_path)
    dist_hospitalization_time = load_dist_from_samples(t0_to_t2_samples_path)

    progression = 1:num_individuals .|> _ -> sample_progression(rng, 
        dist_severity, 
        dist_incubation_time, 
        dist_symptom_onset_time, 
        dist_hospitalization_time)
    
    households = 1:num_individuals |> collect #TODO
    
    params = SimParams(
        households,
        progression,        
        1.0,
        
    )

    params
end

params = initialize(
    population_path="data/simulations/wroclaw-population-orig.csv.gz", 
    incubation_time_samples_path="test/models/assets/incubation_period_distribution.npy", 
    t0_to_t1_samples_path="test/models/assets/t1_distribution.npy",
    t0_to_t2_samples_path="test/models/assets/t1_t2_distribution.npy")




SimParams(Int32[1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  636298, 636299, 636300, 636301, 636302, 636303, 636304, 636305, 636306, 636307], Progression[Progression(Severe, 2.0f0, 3.25f0, 15.0f0), Progression(Asymptomatic, 8.0f0, 10.25f0, NaN32), Progression(Asymptomatic, 8.0f0, 17.25f0, NaN32), Progression(Critical, 3.0f0, 4.25f0, 7.0f0), Progression(Severe, 2.0f0, 2.25f0, 9.0f0), Progression(Mild, 2.0f0, 2.25f0, NaN32), Progression(Critical, 4.0f0, 5.25f0, 8.0f0), Progression(Asymptomatic, 4.0f0, 6.25f0, NaN32), Progression(Severe, 3.0f0, 4.25f0, 5.0f0), Progression(Mild, 1.0f0, 2.25f0, NaN32)  …  Progression(Asymptomatic, 7.0f0, 9.25f0, NaN32), Progression(Asymptomatic, 5.0f0, 5.25f0, NaN32), Progression(Severe, 6.0f0, NaN32, 6.5f0), Progression(Mild, 2.0f0, 2.25f0, NaN32), Progression(Mild, 4.0f0, 4.25f0, NaN32), Progression(Mild, 4.0f0, 6.25f0, NaN32), Progression(Critical, 2.0f0, 2.25f0, 2.5f0), Progression(Asymptomatic, 6.0f0, 6.25f0, NaN32), Progression(Asymptomatic, 3.0f0, 5.25f0, NaN32

In [ ]:



state = SimState(params.households |> length, seed=1024)

push!(state.queue, OutsideInfectionEvent(0.0,1))

@time for i in 1:10^10
    if isempty(state.queue)
        @info "Empty queue"
        break
    end

    event = pop!(state.queue)
    #event |> display
    
    state.time = time(event)

    execute!(state, params, event)
end


In [23]:
sample(MersenneTwister(), 1:5)

5

In [5]:
nothing == nothing

true

In [6]:
function simulate!(state, params)
    while true
        if isempty(queue)
            @info "Empty queue"
            break
        end
        
        event = pop!(state.queue)
        if state.affected_people >= params.stop_simulation_threshold
            @info "The outbreak reached a high number $(params.stop_simulation_threshold)"
            break
        else
            event.time >= params.max_time
            @info "Max time reached"
            break
        end
        
        state.time = time(event)
        
        execute!(state, params, event)
    end
end

simulate! (generic function with 1 method)

In [7]:
#h = DataStructures.BinaryHeap{AbstractEvent, DataStructures.LessThan}()
h = DataStructures.BinaryHeap{AbstractEvent, Earlier}()
push!(h, OutsideInfectionEvent(1.2, 34))
push!(h, OutsideInfectionEvent(1.3, 45))
push!(h, TransmissionEvent(1.25, 100, 56, ConstantKernelContact))

pop!(h) |> display
pop!(h) |> display
pop!(h) |> display

OutsideInfectionEvent(1.2f0, 34)

TransmissionEvent(1.25f0, 100, 56, ConstantKernelContact)

OutsideInfectionEvent(1.3f0, 45)